In [0]:
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline 

# Strassen algorithm



In [0]:
def naive_multiplication(A, B):
    """
    Naive matrix multiplication
    Parameters: Matrices A, B
    
    Returns: Matrix C = AB
    """
    
    rowsA, columnsA = len(A), len(A[0])
    rowsB, columnsB = len(B), len(B[0])
    
    #if columnsA != rowsB:
    #    return "can not multiply" 
    # or throw/return something else
    
    C = np.zeros((rowsA, columnsB))
    for i in range(rowsA):
        for j in range(columnsB):
            for k in range(rowsB):
                C[i][j] += A[i][k] * B[k][j]
    return C

In [0]:
# Implemented using the pseudocodes given in
# https://en.wikipedia.org/wiki/Strassen_algorithm
# http://users.monash.edu/~lloyd/tildeProgLang/Algol68/strassen.a68

def strassen_util(A, B):
    n = len(A)
    if n <= 4:
        # I could use 667 here, but just to let Strassen to be called instead of naive multiplication
        # for more cases considering that you ask to use max(N) = 700 in the below tasks.
        return naive_multiplication(A, B) # no point of using Strassen for smaller matrices
    else:
        half_size = (n >> 1)
        #A11 = np.zeros((half_size, half_size))
        A11 = A[:half_size, :half_size]
        #A12 = np.zeros((half_size, half_size))
        A12 = A[:half_size, half_size:]
        #A21 = np.zeros((half_size, half_size))
        A21 = A[half_size:, :half_size]
        #A22 = np.zeros((half_size, half_size))
        A22 = A[half_size:, half_size:]
        
        #B11 = np.zeros((half_size, half_size))
        B11 = B[:half_size, :half_size]
        #B12 = np.zeros((half_size, half_size))
        B12 = B[:half_size, half_size:]
        #B21 = np.zeros((half_size, half_size))
        B21 = B[half_size:, :half_size]
        #B22 = np.zeros((half_size, half_size))
        B22 = B[half_size:, half_size:]
    
        A_final = np.zeros((half_size, half_size))
        B_final = np.zeros((half_size, half_size))
          
        p1 = strassen_util(A11 + A22, B11 + B22)
        p2 = strassen_util(A21 + A22, B11)
        p3 = strassen_util(A11, B12 - B22)
        p4 = strassen_util(A22, B21 - B11)
        p5 = strassen_util(A11 + A12, B22)
        p6 = strassen_util(A21 - A11, B11 + B12)
        p7 = strassen_util(A12 - A22, B21 + B22)

        C12 = p3 + p5
        C21 = p2 + p4
        A_final = p1 + p4
        B_final = A_final + p7
        C11 = B_final - p5
        A_final = p1 + p3
        B_final = A_final + p6
        C22 = B_final - p2
        C = np.zeros((n, n))
        #pdb.set_trace()
        C[:half_size, :half_size] = C11[:half_size, :half_size]
        C[:half_size, half_size:] = C12[:half_size, :half_size]
        C[half_size:, :half_size] = C21[:half_size, :half_size]
        C[half_size:, half_size:] = C22[:half_size, :half_size]
#         for i in range(half_size):
#             for j in range(half_size):
#                 C[i][j] = C11[i][j]
#                 C[i][j + half_size] = C12[i][j]
#                 C[i + half_size][j] = C21[i][j]
#                 C[i + half_size][j + half_size] = C22[i][j]
        return C
        
    
def strassen(A, B):
    """
    Strassen algorithm for matrix multiplication 
    
    Parameters: Matrices A, B
    
    Returns: Matrix C = AB
    """
    
    rowsA, columnsA = len(A), len(A[0])
    rowsB, columnsB = len(B), len(B[0])
    
    nextPowerOfTwo = 1
    while nextPowerOfTwo <= max(rowsA, rowsB, columnsB):
        nextPowerOfTwo <<= 1
        
    A_expand = np.zeros((nextPowerOfTwo, nextPowerOfTwo))
    B_expand = np.zeros((nextPowerOfTwo, nextPowerOfTwo))
    A_expand[:rowsA, :columnsA] += A
    B_expand[:rowsB, :columnsB] += B 
    C_expand = strassen_util(A_expand, B_expand)
    C = C_expand[:rowsA, :columnsB]
    
    return C

In [4]:
# Just verification of the implementation
# No direct connection with the task

#A = np.array([[1, 2, 3], [4, 5, 6], [0, 1, 2], [5, -1, 3]])
#B = np.array([[4, 2, -3, 6], [5, -1, 0, 4], [-1, 0, 1, -2]])
A = np.array([[1, 2, 3, 4, 5, 6], [4, 5, 6, 5, 6, 10], 
              [0, 1, 2, -1, 0, 2], [5, -1, 3, 4, 5, 6], [5, 3, -2, 10, 5, 6]])
B = np.array([[4, 2, -3, 6, 4], [5, -1, 0, 4, 3], [-1, 0, 1, -2, 2], [10, 9, 8, 7, 6], 
             [-1, 4, 3, -2, 3], [8, 6, 4, 2, 1]])
print(naive_multiplication(A, B))
print(strassen(A, B))
print(np.matmul(A, B))
print(naive_multiplication(A, B) == strassen(A, B))
print(naive_multiplication(A, B) == np.matmul(A, B))

[[ 94.  92.  71.  38.  61.]
 [159. 132.  92.  75. 101.]
 [  9.   2.   2.  -3.   3.]
 [ 95. 103.  59.  50.  68.]
 [180. 153. 102. 118. 106.]]
[[ 94.  92.  71.  38.  61.]
 [159. 132.  92.  75. 101.]
 [  9.   2.   2.  -3.   3.]
 [ 95. 103.  59.  50.  68.]
 [180. 153. 102. 118. 106.]]
[[ 94  92  71  38  61]
 [159 132  92  75 101]
 [  9   2   2  -3   3]
 [ 95 103  59  50  68]
 [180 153 102 118 106]]
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


Comparison of naive matrix multiplication, Strassen's algorithm and the builtin function from numpy:

In [0]:
import time

In [0]:
from tqdm import tqdm

In [0]:
# For simplicty, I will use randomly generated square matrices...
# Since Strassen's algorithm extends the matrix to a 2^n-2^n matrix, I will use only this type 
# of matrices for better visualization.

steps = 8
rng = np.arange(100, 701, 100) #2 ** np.arange(steps)
naive_elapsed = np.zeros(steps)
strassen_elapsed = np.zeros(steps)
numpy_elapsed = np.zeros(steps)

ind = 0
for st in tqdm(rng):
    matrix_size = st
    A = np.random.randint(0xfffff, size = (matrix_size, matrix_size))
    B = np.random.randint(0xfffff, size = (matrix_size, matrix_size))
    t = time.time()
    C = naive_multiplication(A, B)
    naive_elapsed[ind] = time.time() - t
    t = time.time()
    C = strassen(A, B)
    strassen_elapsed[ind] = time.time() - t
    t = time.time()
    C = np.matmul(A, B)
    numpy_elapsed[ind] = time.time() - t
    ind += 1

 29%|██▊       | 2/7 [00:20<00:36,  7.22s/it]

In [0]:
x_axis = rng
plt.loglog(x_axis, naive_elapsed[:steps - 1], label="naive method elapsed time", basex=2)
plt.loglog(x_axis, strassen_elapsed[:steps - 1], label="strassen elasped time", basex=2)
plt.loglog(x_axis, numpy_elapsed[:steps - 1], label="numpy elapsed time", basex=2)
plt.legend()
plt.title("Performance comparison of 3 different methods in logscale")
plt.show()

__Probably, due to almost pure C implementation, numpy works this fast in above examples.__

Since $\mathcal{O}(N^3)$ is exponentially larger than $\mathcal{O}(N^{log2(7)}))$ we can easily conclude that for larger sized matrices, Strassen's algorithm will outperform the naive one. One remark we should make here is that the memory consumed by Strassen is much bigger than the naive one (a bunch of square matrices are created during the runtime).

In [0]:
expected_Naive = np.arange(100, 701, 100) ** 3
expected_Strassen = np.arange(100, 701, 100) ** math.log2(7.)
plt.plot(np.arange(100, 701, 100), expected_Naive, label="Naive")
plt.plot(np.arange(100, 701, 100), expected_Strassen, label="Strassen")
plt.legend(loc=(1.05, 0.5))
plt.title("Naive vs Strassen's method on total operations", loc='right')
plt.show()